# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
#call csv from weatherpy
cities = pd.read_csv("../output_data/cities.csv")
cities.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Wadena,46.44,-95.14,75.2,50,1,4.47,US,1595285337
1,1,Mataura,-46.19,168.86,39.0,83,76,3.00,NZ,1595284757
2,2,Tuktoyaktuk,69.45,-133.04,42.8,81,90,9.17,CA,1595285319
3,3,Necochea,-38.55,-58.74,52.0,94,100,15.99,AR,1595285338
4,4,Bluff,-46.60,168.33,39.0,83,93,3.00,NZ,1595285338


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#configure gmaps and set locations and humidity
gmaps.configure(api_key=g_key)
locations = cities[["Lat", "Lng"]]
humidity = cities["Humidity"].astype(float)

In [9]:
#create heat map from gmaps
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
#display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#create filter data from for "perfect weather"
narrow_city=cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0) & (cities["Wind Speed"] < 10), :] 
narrow_city=narrow_city.dropna(how='any')
#display dataframe
narrow_city.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,Vaini,-21.20,-175.20,75.20,64,0,9.17,TO,1595285341
86,86,Abu Samrah,35.30,37.18,74.35,49,0,8.52,SY,1595285350
137,137,Bela Vista,-22.11,-56.52,74.62,44,0,4.72,BR,1595285357
172,172,Crotone,39.09,17.12,73.44,56,0,5.88,IT,1595285362
214,214,Bāneh,36.00,45.89,71.08,28,0,5.44,IR,1595285368


In [13]:
#get a count of cities
narrow_city.count()

Unnamed: 0    19
City          19
Lat           19
Lng           19
Max Temp      19
Humidity      19
Cloudiness    19
Wind Speed    19
Country       19
Date          19
dtype: int64

In [14]:
#add column to data frame for hotel name
narrow_city["Hotel Name"]= ""
narrow_city.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,26,Vaini,-21.20,-175.20,75.20,64,0,9.17,TO,1595285341,
86,86,Abu Samrah,35.30,37.18,74.35,49,0,8.52,SY,1595285350,
137,137,Bela Vista,-22.11,-56.52,74.62,44,0,4.72,BR,1595285357,
172,172,Crotone,39.09,17.12,73.44,56,0,5.88,IT,1595285362,
214,214,Bāneh,36.00,45.89,71.08,28,0,5.44,IR,1595285368,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:

hotels = []
#url for API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in narrow_city.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "type": "lodging",
        "key": g_key,}

    #print results from places api    
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    #extract results
    results = response['results']
   
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        
        narrow_city.loc[index, "Hotel Name"]= results[0]["name"]
        
        
    except (KeyError, IndexError):
        print("Nope... skipping.")
        


Retrieving Results for Index 26: Vaini.
Closest Vaini hotel is Keleti Beach Resort.
Retrieving Results for Index 86: Abu Samrah.
Nope... skipping.
Retrieving Results for Index 137: Bela Vista.
Closest Bela Vista hotel is Bella Vista Palace Hotel.
Retrieving Results for Index 172: Crotone.
Closest Crotone hotel is Helios Hotel.
Retrieving Results for Index 214: Bāneh.
Closest Bāneh hotel is ساختمان زیتون.
Retrieving Results for Index 217: Semnan.
Closest Semnan hotel is Semnan Tourist Hotel.
Retrieving Results for Index 229: Misratah.
Closest Misratah hotel is فندق الماسة.
Retrieving Results for Index 287: Dawlatabad.
Closest Dawlatabad hotel is WASIM ASGAR KHAN.
Retrieving Results for Index 298: Durusu.
Closest Durusu hotel is Çınar Pansiyon.
Retrieving Results for Index 301: Thasos.
Closest Thasos hotel is To Valsamo.
Retrieving Results for Index 344: Nola.
Closest Nola hotel is Pamaran Hotel.
Retrieving Results for Index 406: Rondonópolis.
Closest Rondonópolis hotel is Serras Hotel.


In [19]:
#display cities with hotel names
narrow_city.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
26,26,Vaini,-21.20,-175.20,75.20,64,0,9.17,TO,1595285341,Keleti Beach Resort
86,86,Abu Samrah,35.30,37.18,74.35,49,0,8.52,SY,1595285350,Suruj
137,137,Bela Vista,-22.11,-56.52,74.62,44,0,4.72,BR,1595285357,Bella Vista Palace Hotel
172,172,Crotone,39.09,17.12,73.44,56,0,5.88,IT,1595285362,Helios Hotel
214,214,Bāneh,36.00,45.89,71.08,28,0,5.44,IR,1595285368,ساختمان زیتون


In [20]:
#count results
narrow_city.count()

Unnamed: 0    19
City          19
Lat           19
Lng           19
Max Temp      19
Humidity      19
Cloudiness    19
Wind Speed    19
Country       19
Date          19
Hotel Name    19
dtype: int64

In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_city.iterrows()]
locations = narrow_city[["Lat", "Lng"]]

In [22]:
#create heatmap with markers for the hotel locations
fig = gmaps.figure()
markers = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)    
fig.add_layer(markers)
#display
fig


Figure(layout=FigureLayout(height='420px'))